In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import json

## MLM para choice
{'lr_mlm': 1e-05, 'mlm_threshold': 0.6000000000000001, 'model_size': 'mini', 'n_epochs_mlm': 10, 'wd_mlm': 0.01}

In [ ]:
filedir = 'models_mlm_mini/' #models_mlm_base, models_mlm_large
os_list = os.listdir(filedir)
for file in os_list:
    df_record = pd.read_csv(filedir+file+'/record.csv', index_col = 0)
    dict_config = json.load(open(filedir+file+'/config.json'))
    print(dict_config)
    print(min(df_record['valid_loss']))
    plt.figure()
    plt.plot(df_record['glob_cnt'], df_record['train_loss'], color = 'b')
    plt.plot(df_record['glob_cnt'], df_record['valid_loss'], color = 'r')
    plt.show()

In [ ]:
filedir = 'models_cls_baseline_mini/' #models_mlm_base, models_mlm_large
os_list = os.listdir(filedir)
for file in os_list:
    df_record = pd.read_csv(filedir+file+'/record.csv', index_col = 0)
    dict_config = json.load(open(filedir+file+'/config.json'))
    print(dict_config)
    print(min(df_record['valid_loss']))
    plt.figure()
    plt.plot(df_record['glob_cnt'], df_record['train_loss'], color = 'b')
    plt.plot(df_record['glob_cnt'], df_record['valid_loss'], color = 'r')
    plt.show()

# TEST on different number of examples

In [13]:
import torch
from torch.optim import AdamW
from transformers import BertTokenizer, BertForSequenceClassification
from transformers.adapters import BertAdapterModel
import pandas as pd
import sys
sys.path.append("..")
from configuration import CONSTANTS as C
from dataset_mlm_cls import MLMDateset, CLSDataset
from configuration import Configuration
from tqdm import tqdm
import os
import time
from train import evaluate, init_tokenizer_model

def test_baseline_cls(config):
    TEST_MODEL_DIR = 'models_cls_baseline_'+ config.model_size + '/' +config.test_model_dir +'/'
    
    tokenizer, model_cls_test = init_tokenizer_model(config)

    if config.baseline_with_adapter == 'yes':
        # model_cls_test.load_adapter(adapter_name_or_path=TEST_MODEL_DIR + 'cls_adapter/', load_as = 'cls_adapter', set_active = True)
        # model_cls_test.load_head(save_directory = TEST_MODEL_DIR + 'cls_adapter_head/', load_as = 'cls')
        model_cls_test.add_adapter('cls_adapter', set_active = True)
        model_cls_test.add_classification_head('cls')
    elif config.baseline_with_adapter == 'no':
        # model_cls_test.load_head(save_directory = TEST_MODEL_DIR + 'cls_adapter_head/', load_as = 'cls')
        model_cls_test.add_classification_head('cls')
    model_cls_test.load_state_dict(torch.load(TEST_MODEL_DIR+'state_dict.pth'))
    model_cls_test.to(C.DEVICE)

    testset_cls = CLSDataset(test_cls, tokenizer)
    testloader_cls = torch.utils.data.DataLoader(testset_cls, batch_size = 16, shuffle = True)

    test_loss = evaluate(model_cls_test, testloader_cls)
    print(test_loss)
    return test_loss

def test_enhanced_cls(config):
    print(C.DEVICE)
    TEST_MODEL_DIR = 'models_cls_enhanced_'+ config.model_size + '/' +config.test_model_dir +'/'
    print(TEST_MODEL_DIR)
    tokenizer, model_cls_test = init_tokenizer_model(config)
    
    model_cls_test.add_adapter('cls_adapter', set_active = True)
    model_cls_test.add_classification_head('cls')
    model_cls_test.load_state_dict(torch.load(TEST_MODEL_DIR+'state_dict.pth'))
    model_cls_test.to(C.DEVICE)

    test_data = pd.read_csv('data/slang_test.csv')
    test_group = test_data.groupby('count')
    key_list = []
    test_loss_list = []
    for key, df_group in test_group:
        testset_cls = CLSDataset(df_group, tokenizer)
        testloader_cls = torch.utils.data.DataLoader(testset_cls, batch_size = 16, shuffle = True)
        test_loss = evaluate(model_cls_test, testloader_cls)
        print(key, test_loss)
        key_list.append(key)
        test_loss_list.append(test_loss)
    return key_list, test_loss_list

In [14]:
import pandas as pd
model_size = 'mini'
model_dir = 'model_1657655623'

dict_test = {'model_size': model_size, 'test_model_dir': model_dir}
config = Configuration(dict_test)
test_enhanced_cls(config)


    

Some weights of the model checkpoint at prajjwal1/bert-mini were not used when initializing BertAdapterModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory